# Domain Generation Algorithm (DGA) Detection

## Table of Contents
* Introduction
* Data Importing
* Data Preprocessing
* Training and Evaluation
* Inference
* Conclusion

## Introduction
[Domain Generation Algorithms](https://en.wikipedia.org/wiki/Domain_generation_algorithm) (DGAs) are used to generate domain names that can be used by the malware to communicate with the command and control servers. IP addresses and static domain names can be easily blocked, and a DGA provides an easy method to generate a large number of domain names and rotate through them to circumvent traditional block lists. We will use a type of recurrent neural network called the [Gated Recurrent Unit](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) (GRU) for this example. This implementation enables users to train their models with up-to-date domain names representative of both benign and DGA generated strings. This capability could also be used in production. This notebook provides a view into the data science workflow to create a DGA detection implementation.

In [1]:
import os
import cudf
import cupy as cp
import torch
import requests
import logging
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, average_precision_score
from dga_detector import DGADetector
from dataloader import DataLoader
from dga_dataset import DGADataset
from utils import str2ascii
from cuml.model_selection import train_test_split

#### Enable console logging

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)

#### Load Input Dataset to GPU Dataframe

In [3]:
INPUT_CSV = "../datasets/benign_and_dga_domains.csv"

In [4]:
gdf = cudf.read_csv(INPUT_CSV)

In [5]:
train_data = gdf['domain']
labels = gdf['type']

Because we have only benign and DGA (malicious) categoriesm, the number of domain types need to be set to 2 (`N_DOMAIN_TYPE=2`). Vocabulary size(`CHAR_VOCAB`) is set to 128 ASCII characters. The values below set for `HIDDEN_SIZE`, `N_LAYERS` of the network, and the `LR` (Learning Rate) give an optimum balance for the network size and performance. They might need be set via experiments when working with other datasets.

In [6]:
LR = 0.001
N_LAYERS = 3
CHAR_VOCAB = 128
HIDDEN_SIZE = 100
N_DOMAIN_TYPE = 2

#### Instantiate DGA detector
Now that the data is ready, the datasets are created, and we've set the parameters for the model, we can use the DGA detector method to create and train the model.

In [7]:
dd = DGADetector(lr=LR)
dd.init_model(n_layers=N_LAYERS, char_vocab=CHAR_VOCAB, hidden_size=HIDDEN_SIZE, n_domain_type=N_DOMAIN_TYPE)

2023-05-09 20:34:04,043 [INFO] Found GPU's now setting up cuda for the model


In [8]:
EPOCHS = 25
TRAIN_SIZE = 0.7
BATCH_SIZE = 10000
MODELS_DIR = 'models'

### Training and Evaluation
Now we train and evaluate the model.

In [9]:
%%time
dd.train_model(train_data, labels, batch_size=BATCH_SIZE, epochs=EPOCHS, train_size=0.7)

2023-05-09 20:34:06,137 [INFO] Initiating model training ...
2023-05-09 20:34:06,138 [INFO] Truncate domains to width: 100
/opt/conda/envs/mor_exp/lib/python3.8/site-packages/cudf/core/column/string.py:3563: FutureWarning: The expand parameter is deprecated and will be removed in a future version. Set expand=False to match future behavior.
  warnings.warn(
Epoch:   0%|          | 0/25 [00:00<?, ?it/s]2023-05-09 20:34:09,961 [INFO] [100000/1433083 (7%)]	Loss: 6696.85
2023-05-09 20:34:12,011 [INFO] [200000/1433083 (14%)]	Loss: 4986.70
2023-05-09 20:34:13,948 [INFO] [300000/1433083 (21%)]	Loss: 3919.60
2023-05-09 20:34:15,894 [INFO] [400000/1433083 (28%)]	Loss: 3367.74
2023-05-09 20:34:17,716 [INFO] [500000/1433083 (35%)]	Loss: 2986.46
2023-05-09 20:34:19,411 [INFO] [600000/1433083 (42%)]	Loss: 2571.78
2023-05-09 20:34:21,034 [INFO] [700000/1433083 (49%)]	Loss: 2257.08
2023-05-09 20:34:22,661 [INFO] [800000/1433083 (56%)]	Loss: 2006.10
2023-05-09 20:34:24,363 [INFO] [900000/1433083 (63%)]

CPU times: user 21min 42s, sys: 25.8 s, total: 22min 8s
Wall time: 11min 52s


### Save Model
Save pretrained model to a given output location.

In [10]:
if not os.path.exists(MODELS_DIR):
    print("Creating directory '{}'".format(MODELS_DIR))
    os.makedirs(MODELS_DIR)

now = datetime.now()
model_filename = "rnn_classifier_{}.bin".format(now.strftime("%Y-%m-%d_%H_%M_%S"))
model_filepath = os.path.join(MODELS_DIR, model_filename)
dd.save_checkpoint(model_filepath)

2023-05-09 20:45:58,176 [INFO] Pretrained model checkpoint saved to location: 'models/rnn_classifier_2023-05-09_20_45_58.bin'


### Inference

Using the model generated above, we now score the test dataset against the model to determine if the domain is likely generated by a DGA or benign.

In [11]:
dga_detector = DGADetector()
dga_detector.load_checkpoint(model_filepath)

domain_train, domain_test, type_train, type_test = train_test_split(gdf, "type", train_size=0.7)
test_df = cudf.DataFrame()
test_df["type"] = type_test.reset_index(drop=True)
test_df["domain"] = domain_test.reset_index(drop=True)

test_dataset = DGADataset(test_df, 100)
test_dataloader = DataLoader(test_dataset, batchsize=BATCH_SIZE)

pred_results = []
true_results = []
for chunk in test_dataloader.get_chunks():
    pred_results.append(list(dga_detector.predict(chunk['domain']).values_host))
    true_results.append(list(chunk['type'].values_host))
pred_results = np.concatenate(pred_results)
true_results = np.concatenate(true_results)
accuracy_score_result = accuracy_score(pred_results, true_results)

print('Model accuracy: %s'%(accuracy_score_result))

2023-05-09 20:45:58,203 [INFO] Found GPU's now setting up cuda for the model


Model accuracy: 0.9927105941427499


In [12]:
average_precision = average_precision_score(true_results, pred_results)

print('Average precision score: {0:0.3f}'.format(average_precision))

Average precision score: 0.981


## Export Model to ONNX

In [13]:
def preprocess(df, pad_max_len):
    df = str2ascii(df[0:32], 'domain')
    df = df.drop("domain", axis=1)
    seq_len_arr = df["len"].values_host
    df = df.drop("len", axis=1)
    seq_len_tensor = torch.LongTensor(seq_len_arr)

    seq_cp = df.to_cupy()
    input = cp.zeros((seq_cp.shape[0], pad_max_len))
    input[:seq_cp.shape[0], :seq_cp.shape[1]] = seq_cp
    input = input.astype("long")
    seq_tensor = torch.as_tensor(input)
    
    if torch.cuda.is_available():
        seq_tensor = seq_tensor.cuda()
        seq_len_tensor = seq_len_tensor.cuda()

    return seq_tensor, seq_len_tensor

In [14]:
input, seq_lengths = preprocess(gdf[0:32], 100)
sample_model_input = (input, seq_lengths)

/opt/conda/envs/mor_exp/lib/python3.8/site-packages/cudf/core/column/string.py:3563: FutureWarning: The expand parameter is deprecated and will be removed in a future version. Set expand=False to match future behavior.
  warnings.warn(


In [15]:
torch.onnx.export(dga_detector.model,              
                  sample_model_input,               
                  "model.onnx",                                      # where to save the model
                  export_params=True,                                # store the trained parameter weights inside the model file
                  opset_version=10,                                  # the ONNX version to export the model to
                  do_constant_folding=True,                          # whether to execute constant folding for optimization
                  input_names = ['domains', "seq_lengths"],          # the model's input names
                  output_names = ['output'],                         # the model's output names
                  dynamic_axes={'domains' : {0 : 'batch_size'},      # variable length axes
                                'seq_lengths': {0: 'batch_size'}, 
                                'output' : {0 : 'batch_size'}})

/my_data/gitrepos/efajardo-nv/morpheus-experimental/dga-detection/training-tuning-inference/rnn_classifier.py:57: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  gru_input = pack_padded_sequence(embedded, seq_lengths.data.cpu().numpy())
/my_data/gitrepos/efajardo-nv/morpheus-experimental/dga-detection/training-tuning-inference/rnn_classifier.py:57: UserWarning: pack_padded_sequence has been called with a Python list of sequence lengths. The tracer cannot track the data flow of Python values, and it will treat them as constants, likely rendering the trace incorrect for any other combination of lengths.
  gru_input = pack_padded_sequence(embedded, seq_lengths.data.cpu().numpy())
/opt/conda/envs/mor_exp/lib/python3.8/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWar

## Conclusion

This DGA detection implementation enables users to train their models for detection and also use existing models. This capability could also be used in conjunction with log parsing efforts if the logs contain domain names. Data is kept in GPU memory, removing unnecessary copy/converts and providing a 4X speed advantage over CPU only implementations. This is esepcially true with large batch sizes.